# 限制控制智能体执行

In [1]:
from langchain_openai import ChatOpenAI
import os

api_key = os.getenv("LOCAL_API_KEY")
base_url = os.getenv("LOCAL_API_BASE")

model = ChatOpenAI(api_key=api_key, 
                   base_url=base_url,
                   temperature=0.3, 
                   max_tokens=4096, 
                )

In [4]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

promptTemplate = """尽可能的帮助用户回答任何问题。
您可以使用以下工具来帮忙解决问题，如果已经知道答案，也可以直接回答：
{tools}

回复格式说明：
----------------------------------

回复我时，请以以下两种格式之一输出问题：

选项1：如果您希望人类使用工具，请使用此选项。
采用以下JSON模式格式化的回复内容：
```json
{{
    "reason": string, \\ 叙述使用工具的原因
    "action": string, \\ 要使用的工具。 必须是 {tool_names} 之一
    "action_input": string \\ 工具的输入
}}
```

选项2：如果您认为你自己已经有了答案或者已经通过使用工具找到了答案，想直接对人类做出反应，请使用此选项。
采用以下JSON模式格式化的回复内容：
```json
{{
    "action": "Final Answer",
    "answer": string \\ 最终回复问题的答案放在这里！
}}
```

用户输入
----------------------------------
这是用户的输入（请记住通过单个选项，以JSON模式格式化的回复内容，不要回复其他内容）：
{input}
"""
promptTemplate

'尽可能的帮助用户回答任何问题。\n您可以使用以下工具来帮忙解决问题，如果已经知道答案，也可以直接回答：\n{tools}\n\n回复格式说明：\n----------------------------------\n\n回复我时，请以以下两种格式之一输出问题：\n\n选项1：如果您希望人类使用工具，请使用此选项。\n采用以下JSON模式格式化的回复内容：\n```json\n{{\n    "reason": string, \\ 叙述使用工具的原因\n    "action": string, \\ 要使用的工具。 必须是 {tool_names} 之一\n    "action_input": string \\ 工具的输入\n}}\n```\n\n选项2：如果您认为你自己已经有了答案或者已经通过使用工具找到了答案，想直接对人类做出反应，请使用此选项。\n采用以下JSON模式格式化的回复内容：\n```json\n{{\n    "action": "Final Answer",\n    "answer": string \\ 最终回复问题的答案放在这里！\n}}\n```\n\n用户输入\n----------------------------------\n这是用户的输入（请记住通过单个选项，以JSON模式格式化的回复内容，不要回复其他内容）：\n{input}\n'

In [5]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "你是非常强大的助手，你可以使用各种工具来完成人类交给的问题和任务。"),
    ("user", promptTemplate),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])
prompt


ChatPromptTemplate(input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'], input_types={'agent_scratchpad': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMess

In [6]:
from langchain.agents.json_chat.prompt import TEMPLATE_TOOL_RESPONSE

TEMPLATE_TOOL_RESPONSE

"TOOL RESPONSE: \n---------------------\n{observation}\n\nUSER'S INPUT\n--------------------\n\nOkay, so what is the response to my last comment? If using information obtained from the tools you must mention it explicitly without mentioning the tool names - I have forgotten all TOOL RESPONSES! Remember to respond with a markdown code snippet of a json blob with a single action, and NOTHING else - even if you just want to respond to the user. Do NOT respond with anything except a JSON snippet no matter what!"

In [7]:
TEMPLATE_TOOL_RESPONSE = """工具响应：
----------------------------------

{observation}

用户的输入：
----------------------------------
请根据工具的响应判断，是否能够回答问题：

{input}

请根据工具响应的内容，思考接下来的回复。回复格式严格按照前面所说的2种JSON回复格式，
选择其中1种进行回复。请记住通过单个选项，以JSON模式格式化回复内容，不要回复其他内容。
"""

In [9]:
from langchain_core.messages import HumanMessage, BaseMessage, AIMessage
from typing import List


def format_log_to_messages(
    query,
    intermediate_steps,
    template_tool_response
):
    """构建思考过程"""
    thoughts: List[BaseMessage] = []
    for action, observation in intermediate_steps:
        thoughts.append(AIMessage(content=action.log))
        human_message = HumanMessage(
            content=template_tool_response.format(
                input=query,
                observation=observation
            )
        )
        thoughts.append(human_message)
    return thoughts

In [10]:
from langchain.agents import tool

@tool
def word_length(word: str) -> int:
    """计算传入的单词的长度"""
    return len(word)

word_length.invoke("hello")

5

In [15]:
import requests

@tool
def get_weather(location: str) -> dict:
    """根据城市获取天气数据"""
    api_key = "SKcA5FGgmLvN7faJi"
    city_codes = {
        "西安": "xian",
        "北京": "beijing",
        "上海": "shanghai",
        "广州": "guangzhou",
        "深圳": "shenzhen",
        "杭州": "hangzhou",
        "成都": "chengdu",
        "武汉": "wuhan"
    }
    
    # 处理城市名称
    location = location.replace("市", "").replace("区", "")
    location = city_codes.get(location, location)  # 如果有映射就用映射，没有就用原名
    
    
    url = f"https://api.seniverse.com/v3/weather/now.json?key={api_key}&location={location}&language=zh-Hans&unit=c"
    try:
        response = requests.get(url)
        #print(location)
        if response.status_code == 200:
            data = response.json()
            #print(data)
            weather = {
                'description':data['results'][0]["now"]["text"],
                'temperature':data['results'][0]["now"]["temperature"]
            }
            return weather
        else:
            raise Exception(f"失败接收天气信息：{response.status_code}")
    except Exception as e:
        return {"description": "获取天气信息失败", "detail": str(e)}
            
get_weather.invoke("西安")

{'description': '阴', 'temperature': '3'}

In [16]:
tools = [word_length, get_weather]
tools

[StructuredTool(name='word_length', description='计算传入的单词的长度', args_schema=<class 'langchain_core.utils.pydantic.word_length'>, func=<function word_length at 0x124d78c10>),
 StructuredTool(name='get_weather', description='根据城市获取天气数据', args_schema=<class 'langchain_core.utils.pydantic.get_weather'>, func=<function get_weather at 0x12537fb50>)]

In [17]:
from langchain.tools.render import render_text_description

prompt = prompt.partial(
    tools=render_text_description(tools),
    tool_names=", ".join([t.name for t in tools])
)
prompt


ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'agent_scratchpad': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')], typing.Ann

In [18]:
from langchain.agents.agent import AgentOutputParser
from langchain_core.output_parsers.json import parse_json_markdown
from langchain_core.exceptions import OutputParserException
from langchain_core.agents import AgentAction, AgentFinish

import logging

logger = logging.getLogger(__name__)

class JSONAgentOutputParser(AgentOutputParser):
    
    def parse(self, text: str) -> AgentAction | AgentFinish:
        try:
            response = parse_json_markdown(text)
            if isinstance(response, AgentAction):
                logger.warning(f"Got multiple action responses: {response}")
                response = response[0]
            if response.get("action") == "Final Answer":
                return AgentFinish(return_values={"output": response.get("answer")}, log=text)
            else:
                return AgentAction(tool=response.get("action"), tool_input=response.get("action_input"), log=text)
        except OutputParserException as e:
            raise OutputParserException(f"Failed to parse output: {e}")
        
    
    @property
    def _type(self) -> str:
        return "json_agent"
    


In [19]:
from langchain_core.runnables import RunnablePassthrough

agent = (
    RunnablePassthrough.assign(
        agent_scratchpad=lambda x: format_log_to_messages(
            x["input"], 
            x["intermediate_steps"], 
            template_tool_response = TEMPLATE_TOOL_RESPONSE
        )
    )
    | prompt
    | model
    | JSONAgentOutputParser()
)
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_log_to_messages(x['input'], x['intermediate_steps'], template_tool_response=TEMPLATE_TOOL_RESPONSE))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'agent_scratchpad': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langc

In [22]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)

agent_executor.invoke({"input": '你好，今天xian的天气怎么样？' if '西安' in '你好，今天西安的天气怎么样？' else '你好，今天北京的天气怎么样？'})



> Entering new AgentExecutor chain...
```json
{
    "reason": "获取Xian的当前天气",
    "action": "get_weather",
    "action_input": "西安"
}
```{'description': '阴', 'temperature': '3'}```json
{
    "action": "Final Answer",
    "answer": "今天西安的天气是阴天，温度为3度。"
}
```

> Finished chain.


{'input': '你好，今天xian的天气怎么样？', 'output': '今天西安的天气是阴天，温度为3度。'}

In [23]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, return_intermediate_steps=True)

agent_executor.invoke({"input": '你好，今天xian的天气怎么样？' if '西安' in '你好，今天西安的天气怎么样？' else '你好，今天北京的天气怎么样？'})



> Entering new AgentExecutor chain...
```json
{
    "reason": "获取Xian的当前天气数据",
    "action": "get_weather",
    "action_input": "Xian"
}
```{'description': '阴', 'temperature': '3'}```json
{
    "action": "Final Answer",
    "answer": "今天Xian的天气是阴天，温度为3°C。"
}
```

> Finished chain.


{'input': '你好，今天xian的天气怎么样？',
 'output': '今天Xian的天气是阴天，温度为3°C。',
 'intermediate_steps': [(AgentAction(tool='get_weather', tool_input='Xian', log='```json\n{\n    "reason": "获取Xian的当前天气数据",\n    "action": "get_weather",\n    "action_input": "Xian"\n}\n```'),
   {'description': '阴', 'temperature': '3'})]}

In [24]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, max_iterations=2)

agent_executor.invoke({"input": '你好，今天xian的天气怎么样？' if '西安' in '你好，今天西安的天气怎么样？' else '你好，今天北京的天气怎么样？'})



> Entering new AgentExecutor chain...
```json
{
    "reason": "获取Xian的当前天气数据",
    "action": "get_weather",
    "action_input": "Xian"
}
```{'description': '阴', 'temperature': '3'}```json
{
    "action": "Final Answer",
    "answer": "今天Xian的天气是阴天，温度为3度。"
}
```

> Finished chain.


{'input': '你好，今天xian的天气怎么样？', 'output': '今天Xian的天气是阴天，温度为3度。'}

In [28]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, max_execution_time=10)

agent_executor.invoke({"input": '你好，今天xian的天气怎么样？' if '西安' in '你好，今天西安的天气怎么样？' else '你好，今天北京的天气怎么样？'})



> Entering new AgentExecutor chain...
```json
{
    "reason": "获取Xian当前的天气数据",
    "action": "get_weather",
    "action_input": "Xi'an"
}
```{'description': '获取天气信息失败', 'detail': '失败接收天气信息：404'}```json
{
    "action": "Final Answer",
    "answer": "对不起，我无法获取Xi'an的天气信息。"
}
```

> Finished chain.


{'input': '你好，今天xian的天气怎么样？', 'output': "对不起，我无法获取Xi'an的天气信息。"}